# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902051


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:01<00:24,  1.18it/s]

Rendering models:  13%|█▎        | 4/31 [00:06<00:36,  1.37s/it]

Rendering models:  16%|█▌        | 5/31 [00:06<00:25,  1.00it/s]

Rendering models:  19%|█▉        | 6/31 [00:07<00:18,  1.35it/s]

...

Rendering models:  90%|█████████ | 28/31 [00:09<00:00,  7.96it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  6.79it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  7.23it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  7.67it/s]

equidad1                              0.000249
mistely                               0.000213
nickoftona                            0.000254
ancon                                 0.001019
not-logged-in-203c06289b3e5cb8fb85    0.022612
Drew_Farnsworth                       0.000211
Thedunkmasta                          0.002713
OregonHiker                           0.000391
DrJackie                              0.070557
not-logged-in-7526601586eaa938b3ee    0.000643
lsautter                              0.000139
ImArobot                              0.006843
not-logged-in-3c933f8fe1f845a04fdf    0.017908
not-logged-in-a62efc8bbed690435003    0.297203
earthling001                          0.000085
coyotes54                             0.000090
magedghoche                           0.002054
mclainjamiel                          0.000307
databanana                            0.000169
Planetme                              0.001182
crush202020                           0.000591
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())